In [1]:
from summarizer.domain.model.video import Video
import matplotlib

2022-03-24 21:19:46.593502: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2022-03-24 21:19:47.238779: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-03-24 21:19:47.239275: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2022-03-24 21:19:47.271982: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-24 21:19:47.272255: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:2b:00.0 name: NVIDIA GeForce RTX 3070 computeCapability: 8.6
coreClock: 1.77GHz coreCount: 46 deviceMemorySize: 7.77GiB deviceMemoryBandwidth: 417.29GiB/s
2022-03-24 21:19:47.272271: I tensorflow/stream_executor/platform/def

1.1942145824432373


In [ ]:

import tensorflow as tf
gpu = tf.config.experimental.list_physical_devices('GPU') # 내 컴에 장착된 GPU를 list로 반환
try:
    tf.config.experimental.set_memory_growth(gpu[0], True) # GPU Memory Growth를 Enable
except RuntimeError as e:
    print(e) # Error 발생하면 Error 내용 출력

In [ ]:
video = Video(url="tests/2.mp4")
images = video._read_video()
idx, image = next(images)
matplotlib.pyplot.imshow(image.frame)

In [ ]:

features = image.extract()
print(features)
video.shorten(video_feature=features, must_include_feature=["person"])

In [2]:
video = Video(url="tests/2.mp4") 
list_of_feature = video.extract_feature()


length : 4625
width : 640
height : 360
fps : 25.0


2022-03-24 01:47:22.701826: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)


In [3]:
video.shorten(video_feature=list_of_feature, must_include_feature=["person"])

length : 0
width : 0
height : 0
fps : 0.0


In [4]:
video2 = Video(url="tests/2.mp4")

In [ ]:
video2._read_video()

In [5]:
video2.shorten(video_feature=list_of_feature, must_include_feature=["person"])

OpenCV: FFMPEG: tag 0x58564944/'DIVX' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'


length : 4625
width : 640
height : 360
fps : 25.0


In [ ]:
import cv2
video_feature = list_of_feature
must_include_feature = ["person"]
parameter = video2._get_parameter()
images = video2._read_video()
fps = parameter["fps"]
one_sec_images = []
to_concat_timeframe = []
concated_image = []
fourcc = cv2.VideoWriter_fourcc(*'DIVX')
out = cv2.VideoWriter("out.mp4", fourcc, fps, (parameter["width"], parameter["height"]))

for feature in video_feature:
    ch = False
    for x in must_include_feature:
        if x == feature.name:
            ch = True
            break
    if ch:
        to_concat_timeframe.append(feature.current_frame)

print(to_concat_timeframe)
for idx, image in images:
    one_sec_images.append(image)
    if(idx%fps == 0):
        if(idx in to_concat_timeframe):
            concated_image.extend(one_sec_images)
        one_sec_images = []
print(len(concated_image))
for image in concated_image:
    out.write(image.frame)
out.release()

